In [3]:
from langchain_groq import ChatGroq

llm = ChatGroq(
    model="llama-3.1-70b-versatile",
    groq_api_key='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',
    temperature=0,
)

response = llm.invoke("The first person to land on sun was...")
print(response.content)

It's not possible for a person to land on the sun.  The sun is a massive ball of hot, glowing gas and does not have a solid surface. Additionally, the temperatures on the sun are so high (reaching over 5,500°C or 10,000°F) that any living organism or spacecraft would immediately vaporize.


In [4]:
from langchain_community.document_loaders import WebBaseLoader

loader = WebBaseLoader("https://jobs.nike.com/job/R-38721/")
page_data = loader.load().pop().page_content
print(page_data)

Apply for Software Engineer II - Engineering Enablement (Remote Work Option)

Search JobsSkip navigationSearch JobsNIKE, INC. JOBSContract JobsJoin The Talent CommunityLife @ NikeOverviewBenefitsBrandsOverviewJordanConverseTeamsOverviewAdministrative SupportAdvanced InnovationAir Manufacturing InnovationAviationCommunicationsCustomer ServiceDesignDigitalFacilitiesFinance & AccountingGovernment & Public AffairsHuman ResourcesInsights & AnalyticsLegalManufacturing & EngineeringMarketingMerchandisingPlanningPrivacyProcurementProduct Creation, Development & ManagementRetail CorporateRetail StoresSalesSocial & Community ImpactSports MarketingStrategic PlanningSupply Chain, Distribution & LogisticsSustainabilityTechnologyLocationsOverviewNike WHQNike New York HQEHQ: Hilversum, The NetherlandsELC: Laakdal, BelgiumGreater China HQDiversity, Equity & InclusionOverviewMilitary InclusionDisability InclusionIndigenous InclusionInternshipsTechnologySoftware Engineer II - Engineering Enablement (Rem

In [5]:
from langchain_core.prompts import PromptTemplate

prompt_extract = PromptTemplate.from_template(
        """
        ### SCRAPED TEXT FROM WEBSITE:
        {page_data}
        ### INSTRUCTION:
        The scraped text is from the career's page of a website.
        Your job is to extract the job postings and return them in JSON format containing the 
        following keys: `role`, `experience`, `skills` and `description`.
        Only return the valid JSON.
        ### VALID JSON (NO PREAMBLE):    
        """
)

chain_extract = prompt_extract | llm 
res = chain_extract.invoke(input={'page_data':page_data})
type(res.content)

str

In [8]:
import pandas as pd

df = pd.read_csv("my_portfolio.csv")
df

,Techstack,Links
0,"React, Node.js, MongoDB",https://example.com/react-portfolio
1,"Angular,.NET, SQL Server",https://example.com/angular-portfolio
2,"Vue.js, Ruby on Rails, PostgreSQL",https://example.com/vue-portfolio
3,"Python, Django, MySQL",https://example.com/python-portfolio
4,"Java, Spring Boot, Oracle",https://example.com/java-portfolio
5,"Flutter, Firebase, GraphQL",https://example.com/flutter-portfolio
6,"WordPress, PHP, MySQL",https://example.com/wordpress-portfolio
7,"Magento, PHP, MySQL",https://example.com/magento-portfolio
8,"React Native, Node.js, MongoDB",https://example.com/react-native-portfolio
9,"iOS, Swift, Core Data",https://example.com/ios-portfolio


In [6]:
from langchain_core.output_parsers import JsonOutputParser

json_parser = JsonOutputParser()
json_res = json_parser.parse(res.content)
json_res


{'role': 'Software Engineer II - Engineering Enablement',
 'experience': '3+ years of experience in software development building back-end components and services',
 'skills': ['AWS services such as: Lambda, Elasticsearch, Dynamo and Kinesis',
  'NodeJS',
  'Java',
  'Python',
  'Terraform',
  'CloudFormation'],
 'description': 'Design and implement highly scalable cloud-based data services. Develop frameworks that enable data ingestion using various patterns. Utilize continuous integration and deployment frameworks including automated unit tests and integration testing. Contribute to collaborative reviews of designs, code, and test plans. Support existing applications, resolve defects and develop enhancements.'}

In [10]:
import uuid
import chromadb

client = chromadb.PersistentClient('vectorstore')
collection = client.get_or_create_collection(name="portfolio")

if not collection.count():
    for _, row in df.iterrows():
        collection.add(documents=row["Techstack"],
                       metadatas={"links": row["Links"]},
                       ids=[str(uuid.uuid4())])

In [18]:
links = collection.query(query_texts=job['skills'], n_results=2).get('metadatas', [])
links

NameError: name 'job' is not defined

In [20]:
job = json_res
job['skills']


['AWS services such as: Lambda, Elasticsearch, Dynamo and Kinesis',
 'NodeJS',
 'Java',
 'Python',
 'Terraform',
 'CloudFormation']

In [21]:
prompt_email = PromptTemplate.from_template(
        """
        ### JOB DESCRIPTION:
        {job_description}
        
        ### INSTRUCTION:
        You are Mohan, a business development executive at AtliQ. AtliQ is an AI & Software Consulting company dedicated to facilitating
        the seamless integration of business processes through automated tools. 
        Over our experience, we have empowered numerous enterprises with tailored solutions, fostering scalability, 
        process optimization, cost reduction, and heightened overall efficiency. 
        Your job is to write a cold email to the client regarding the job mentioned above describing the capability of AtliQ 
        in fulfilling their needs.
        Also add the most relevant ones from the following links to showcase Atliq's portfolio: {link_list}
        Remember you are Mohan, BDE at AtliQ. 
        Do not provide a preamble.
        ### EMAIL (NO PREAMBLE):
        
        """
        )

chain_email = prompt_email | llm
res = chain_email.invoke({"job_description": str(job), "link_list": links})
print(res.content)

Subject: Expert Software Engineering Services for Your Engineering Enablement Needs

Dear Hiring Manager,

I came across your job posting for a Software Engineer II - Engineering Enablement, and I'm excited to introduce you to AtliQ, a leading AI & Software Consulting company. Our team of experts has extensive experience in designing and implementing highly scalable cloud-based data services, which aligns perfectly with your requirements.

At AtliQ, we have a proven track record of delivering tailored solutions that foster scalability, process optimization, cost reduction, and heightened overall efficiency. Our team is well-versed in AWS services such as Lambda, Elasticsearch, Dynamo, and Kinesis, as well as NodeJS, Java, Python, Terraform, and CloudFormation. We're confident that our expertise can help you achieve your goals.

I'd like to highlight a few relevant projects from our portfolio that demonstrate our capabilities:

* We developed a Machine Learning-based solution using Pyth